In [ ]:
!pip install pandas
!pip install bertopic
!pip install nltk
!pip install cohere
!pip install tiktoken
!pip install tweet-preprocessor


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 3.0 MB/s eta 0:00:00
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
import os
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from bertopic import BERTopic
from nltk.corpus import stopwords
import preprocessor as p

p.set_options(p.OPT.URL)

standard_stop_words = stopwords.words('english')

def clean_text(text):
    text = p.clean(text)

    text = re.sub(r'\d+', '', text)

    words = text.split()
    filtered_words = [word for word in words if word.lower() not in standard_stop_words]

    return ' '.join(filtered_words)

vectorizer_model = TfidfVectorizer(stop_words=standard_stop_words, ngram_range=(1, 3))

model = BERTopic(embedding_model="all-MiniLM-L6-v2", vectorizer_model=vectorizer_model, calculate_probabilities=False, nr_topics="auto")

path = "/content"
all_files = [os.path.join(path, file) for file in os.listdir(path) if file.endswith('.csv')]

for file in all_files:
    df = pd.read_csv(file)
    df = df.dropna(subset=['Simple Text'])
    df['Simple Text'] = df['Simple Text'].apply(clean_text)

    documents = df['Simple Text'].tolist()
    topics, _ = model.fit_transform(documents)

    df['Topic'] = topics
    topic_counts = df['Topic'].value_counts()
    print(f"\nTopics in file: {file}")
    print(topic_counts)

    topic_info = model.get_topic_info()
    for index, row in topic_info.iterrows():
        if row['Topic'] != -1:
            words = model.get_topic(row['Topic'])
            print(f"Topic {row['Topic']}: {len(words)} words, representative words: {[word for word, _ in words]}")

    new_file_name = os.path.basename(file)[:-4] + '_with_topics.csv'
    new_file_path = os.path.join(path, new_file_name)
    df.to_csv(new_file_path, index=False)
    print(f"Processed file saved: {new_file_path}")



Topics in file: /content/Cleaned_Facebook_Content_Aware_t=1_P=1.csv.csv
 0    585
-1    269
 1     97
 2     42
 3     18
 4     17
 5     13
 6     12
 7     12
Name: Topic, dtype: int64
Topic 0: 10 words, representative words: ['vote', 'election', 'make', 'voting', 'elections', 'early', 'midterm', 'let', 'sure', 'ballot']
Topic 1: 10 words, representative words: ['biden', 'election', 'trump', 'elections', 'midterm', 'misinformation', 'arizona', 'midterms', 'midterm elections', 'fraud']
Topic 2: 10 words, representative words: ['rights', 'women', 'abortion', 'reproductive rights', 'abortion rights', 'reproductive', 'let', 'lgbtq', 'abortionrights', 'women rights']
Topic 3: 10 words, representative words: ['crime', 'rates', 'problems', 'crime rates', 'republican', 'republicans', 'tuberville', 'party', 'order', 'violent']
Topic 4: 10 words, representative words: ['florida', 'desantis', 'ron desantis', 'ron', 'florida governor', 'election', 'governor', 'trump', 'governor ron', 'florida 